In [1]:
!pip install transformers datasets
from transformers import AutoTokenizer
from transformers import AdamW, BertTokenizerFast, BertConfig, get_linear_schedule_with_warmup
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.5 MB/s 
     |████████████████████████████████| 452 kB 67.8 MB/s 
     |████████████████████████████████| 182 kB 82.3 MB/s 
     |████████████████████████████████| 7.6 MB 53.8 MB/s 
     |████████████████████████████████| 132 kB 71.1 MB/s 
     |████████████████████████████████| 212 kB 68.3 MB/s 
     |████████████████████████████████| 127 kB 63.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

# Making Swag

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import json
import os
os.makedirs('/content/drive/MyDrive/ntu-adl-hw2-spring-2021/results', exist_ok=True)
train_data = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/train.json'))
valid_data = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/valid.json'))
context = json.load(open('drive/MyDrive/ntu-adl-hw2-spring-2021/context.json'))
print(train_data[0])

Mounted at /content/drive
{'id': '593f14f960d971e294af884f0194b3a7', 'question': '舍本和誰的數據能推算出連星的恆星的質量？', 'paragraphs': [2018, 6952, 8264, 836], 'relevant': 836, 'answer': {'text': '斯特魯維', 'start': 108}}


In [3]:
one, two, three, four = [], [], [], []
label, question = [], []
for i, data in enumerate(train_data):
    one.append(context[data['paragraphs'][0]])
    two.append(context[data['paragraphs'][1]])
    three.append(context[data['paragraphs'][2]])
    four.append(context[data['paragraphs'][3]])
    label.append(data['paragraphs'].index(data['relevant']))
    question.append(data['question'])
    train = dict({'ending0': one,
                 'ending1': two,
                 'ending2': three,
                 'ending3': four,
                 'label': label,
                 'question': question})
                 
one, two, three, four = [], [], [], []
label, question = [], []

for i, data in enumerate(valid_data):
    one.append(context[data['paragraphs'][0]])
    two.append(context[data['paragraphs'][1]])
    three.append(context[data['paragraphs'][2]])
    four.append(context[data['paragraphs'][3]])
    label.append(data['paragraphs'].index(data['relevant']))
    question.append(data['question'])
    valid = dict({'ending0': one,
                 'ending1': two,
                 'ending2': three,
                 'ending3': four,
                 'label': label,
                 'question': question})
from datasets import Dataset
train_dataset = Dataset.from_dict(train)
valid_dataset = Dataset.from_dict(valid)
train_dataset
#valid_dataset

Dataset({
    features: ['ending0', 'ending1', 'ending2', 'ending3', 'label', 'question'],
    num_rows: 21714
})

In [12]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]
def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["question"]]
    second_sentences = [
        [f"{examples[end][i]}" for end in ending_names] for i in range(len(examples["question"]))
    ]
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [13]:
examples = train_dataset[:5]
features = preprocess_function(examples)
print(features['input_ids'][0][0])
# 101 -> CLS, 102 -> SEP
#idx = 0
#[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

[101, 5650, 3315, 1469, 6306, 4638, 3149, 3087, 5543, 2972, 5050, 1139, 6865, 3215, 4638, 2604, 3215, 4638, 6549, 7030, 8043, 102, 9275, 2399, 8024, 1313, 2428, 4289, 4415, 2119, 2157, 5979, 2357, 2861, 7679, 2225, 2993, 185, 7092, 2548, 2861, 1853, 1305, 3418, 3087, 2451, 5412, 4685, 2205, 6316, 6243, 5050, 1139, 6549, 7030, 1920, 3176, 122, 119, 125, 945, 1922, 7382, 6549, 7030, 4638, 7478, 6752, 1240, 3215, 7768, 3298, 1728, 7028, 1213, 1847, 5240, 2768, 4158, 7442, 2094, 5080, 882, 2706, 511, 2695, 672, 7524, 7426, 4197, 1762, 4415, 6316, 677, 3118, 2898, 7946, 3822, 2100, 1762, 4638, 1377, 5543, 2595, 8024, 852, 1398, 3229, 6291, 4158, 7092, 2548, 2861, 1853, 1305, 4638, 6223, 7953, 752, 2179, 677, 679, 5543, 2768, 4989, 8024, 800, 6291, 4158, 519, 2746, 4534, 3300, 3378, 4934, 5632, 4197, 2137, 2526, 7349, 3632, 2604, 3215, 1139, 4412, 6857, 4934, 5774, 1538, 4638, 6121, 4158, 520, 511, 4534, 3229, 4638, 4289, 4415, 2119, 2157, 1963, 3797, 5455, 510, 773, 1164, 185, 5397, 5162, 5

In [16]:
idx = 0
tmp = [tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]
cnt = 0
for t in tmp:
  cnt += len(t)
cnt

2956

In [7]:
train_swag = train_dataset.map(preprocess_function, batched=True)
valid_swag = valid_dataset.map(preprocess_function, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

# Multiple Choice Training

In [8]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained('bert-base-chinese')

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-chinese and are newly

In [9]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [10]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [11]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/ntu-adl-hw2-spring-2021/results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_swag,
    eval_dataset=valid_swag,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ending0, ending2, ending3, question, ending1. If ending0, ending2, ending3, question, ending1 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21714
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 27140


Epoch,Training Loss,Validation Loss,Accuracy
0,0.187200,0.197288,0.947491
1,0.123900,0.273920,0.945829
2,0.058500,0.277380,0.953805
3,0.014900,0.327163,0.956132
4,0.005200,0.359730,0.958126


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: ending0, ending2, ending3, question, ending1. If ending0, ending2, ending3, question, ending1 are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3009
  Batch size = 2
Saving model checkpoint to /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results/checkpoint-5428
Configuration saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results/checkpoint-5428/config.json
Model weights saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results/checkpoint-5428/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results/checkpoint-5428/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ntu-adl-hw2-spring-2021/results/checkpoint-5428/special_tokens_map.json
The following columns in the evaluation set

TrainOutput(global_step=27140, training_loss=0.07792008237620883, metrics={'train_runtime': 25120.8346, 'train_samples_per_second': 4.322, 'train_steps_per_second': 1.08, 'total_flos': 1.118998927983179e+17, 'train_loss': 0.07792008237620883, 'epoch': 5.0})